In [4]:
import os
import json
import openai
from openai import OpenAI
import torch
import networkx as nx
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional, Any
from transformers import AutoTokenizer, AutoModelForCausalLM
from collections import defaultdict
import time

In [ ]:
class LLMPersonaDiscoveryPipeline:
    """
    LLM 간 대화 생성 → 브리징 추론 → 페르소나 발견 파이프라인
    """
    
    def __init__(self, openai_api_key: str, persona_schema_path: str):
        self.openai_api_key = openai_api_key
        openai.api_key = openai_api_key
        
        # 페르소나 스키마 로드
        self.persona_schema = self._load_persona_schema(persona_schema_path)
        
        # 페르소나 카테고리 리스트 (JSON에서 자동 생성)
        self.persona_categories = self._extract_persona_categories()
        
        # 타겟 LLM 모델 리스트
        self.target_llm_models = [
            "deepseek-ai/deepseek-vl2-tiny",
            "meta-llama/Llama-3.2-1B",
            "Qwen/Qwen2.5-0.5B"
        ]
        
        # 로드된 모델 캐시
        self.loaded_models = {}
        
        # 페르소나 그래프
        self.persona_graph = nx.DiGraph()
        
        # 브리징 관계 저장
        self.bridging_relations = []
        
        # GPU 사용 가능 여부
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🖥️  사용 디바이스: {self.device}")
        print(f"📋 로드된 페르소나 카테고리 수: {len(self.persona_categories)}")
    
    def _load_persona_schema(self, schema_path: str) -> Dict[str, Any]:
        """페르소나 스키마 JSON 파일 로드"""
        try:
            with open(schema_path, 'r', encoding='utf-8') as f:
                schema = json.load(f)
            print(f"✅ 페르소나 스키마 로드 완료: {schema_path}")
            return schema
        except FileNotFoundError:
            print(f"⚠️  스키마 파일을 찾을 수 없습니다: {schema_path}")
            return self._get_default_schema()
        except json.JSONDecodeError as e:
            print(f"❌ JSON 파싱 오류: {e}")
            return self._get_default_schema()
    
    def _get_default_schema(self) -> Dict[str, Any]:
        """기본 스키마 반환 (파일 로드 실패 시)"""
        return {
            "structure": {
                "social_role": {
                    "categories": {
                        "professional": {"examples": ["Doctor", "Lawyer"]},
                        "technical_management": {"examples": ["Software Engineer"]},
                        "general_role": {"examples": ["Student", "Artist"]}
                    }
                }
            }
        }
    
    def _extract_persona_categories(self) -> List[str]:
        """JSON 스키마에서 모든 페르소나 카테고리 추출"""
        categories = []
        
        try:
            structure = self.persona_schema.get("structure", {})
            social_role = structure.get("social_role", {})
            role_categories = social_role.get("categories", {})
            
            # 모든 역할 카테고리에서 예시 추출
            for category_key, category_data in role_categories.items():
                examples = category_data.get("examples", [])
                categories.extend(examples)
            
            print(f"📊 추출된 페르소나: {len(categories)}개")
            
            # 중복 제거 및 정렬
            categories = sorted(list(set(categories)))
            
            return categories
            
        except Exception as e:
            print(f"⚠️  페르소나 추출 중 오류: {e}")
            # 기본 페르소나 반환
            return [
                "Doctor", "Lawyer", "Teacher", "Engineer", 
                "Artist", "Writer", "Student", "Chef"
            ]
    
    def get_persona_by_category(self, category_type: str) -> List[str]:
        """특정 카테고리의 페르소나만 반환
        
        Args:
            category_type: 'professional', 'technical_management', 'general_role'
        """
        try:
            structure = self.persona_schema.get("structure", {})
            social_role = structure.get("social_role", {})
            role_categories = social_role.get("categories", {})
            
            category_data = role_categories.get(category_type, {})
            return category_data.get("examples", [])
            
        except Exception as e:
            print(f"⚠️  카테고리 조회 중 오류: {e}")
            return []
    
    def get_all_persona_attributes(self) -> Dict[str, Any]:
        """페르소나의 모든 속성 정보 반환 (성격, 배경, 관심사 등)"""
        return self.persona_schema.get("structure", {})
    
    def print_persona_summary(self):
        """로드된 페르소나 정보 요약 출력"""
        print("\n" + "="*60)
        print("📋 페르소나 스키마 요약")
        print("="*60)
        
        structure = self.persona_schema.get("structure", {})
        
        # 1. 직업/역할
        social_role = structure.get("social_role", {})
        print("\n1️⃣  직업 및 사회적 역할")
        for cat_key, cat_data in social_role.get("categories", {}).items():
            print(f"   • {cat_data.get('name', cat_key)}: {len(cat_data.get('examples', []))}개")
        
        # 2. 성격
        personality = structure.get("personality", {})
        print(f"\n2️⃣  성격 특성 (Big Five): {len(personality.get('categories', []))}개 차원")
        
        # 3. 배경
        background = structure.get("background", {})
        print(f"\n3️⃣  배경 정보: {len(background.get('categories', {}))}개 카테고리")
        
        # 4. 관심사
        interests = structure.get("interests", {})
        print(f"\n4️⃣  선호도 및 관심사: {len(interests.get('categories', {}))}개 카테고리")
        
        print(f"\n✅ 총 페르소나 수: {len(self.persona_categories)}개")
        print("="*60 + "\n")
        
    # ==================== 모델 로딩 ====================

    def load_target_model(self, model_name: str) -> Tuple[AutoTokenizer, AutoModelForCausalLM]:
        """
        허깅페이스 모델 로드
        
        Args:
            model_name: 모델 이름 (예: "meta-llama/Llama-3.2-1B")
        
        Returns:
            (tokenizer, model)
        """
        if model_name in self.loaded_models:
            print(f"✅ 캐시에서 모델 로드: {model_name}")
            return self.loaded_models[model_name]
        
        print(f"📥 모델 다운로드 중: {model_name}")
        
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else None,
                trust_remote_code=True
            )
            
            if self.device == "cpu":
                model = model.to(self.device)
            
            # 패딩 토큰 설정
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
            
            self.loaded_models[model_name] = (tokenizer, model)
            print(f"✅ 모델 로드 완료: {model_name}")
            
            return tokenizer, model
            
        except Exception as e:
            print(f"❌ 모델 로드 실패: {model_name}")
            print(f"   오류: {e}")
            return None, None

    # ==================== STEP 1: 페르소나 부여 ====================

    def _get_persona_category_type(self, persona: str) -> str:
        """페르소나가 속한 카테고리 타입 찾기"""
        structure = self.persona_schema.get("structure", {})
        social_role = structure.get("social_role", {})
        role_categories = social_role.get("categories", {})
        
        for category_key, category_data in role_categories.items():
            if persona in category_data.get("examples", []):
                return category_key
        
        return "general_role"  # 기본값

    def _generate_persona_attributes(self, persona: str) -> Dict[str, Any]:
        """페르소나에 맞는 랜덤 속성 생성"""
        import random
        
        structure = self.persona_schema.get("structure", {})
        
        # 성격 특성 (Big Five) 랜덤 생성
        personality_traits = {}
        for trait_dict in structure.get("personality", {}).get("categories", []):
            for trait_name, trait_info in trait_dict.items():
                personality_traits[trait_name] = random.choice(["high", "low"])
        
        # 배경 정보 랜덤 선택
        background_categories = structure.get("background", {}).get("categories", {})
        background = {}
        for bg_key, bg_data in background_categories.items():
            examples = bg_data.get("examples", [])
            if examples:
                background[bg_key] = random.choice(examples)
        
        # 관심사 랜덤 선택 (3-5개)
        interests_categories = structure.get("interests", {}).get("categories", {})
        interests = {}
        for int_key, int_data in interests_categories.items():
            examples = int_data.get("examples", [])
            if examples and int_key in ["hobbies", "content_preferences"]:
                # 취미와 콘텐츠 선호도는 2-3개 선택
                num_choices = min(random.randint(2, 3), len(examples))
                interests[int_key] = random.sample(examples, num_choices)
            elif examples:
                # 나머지는 1개만 선택
                interests[int_key] = random.choice(examples)
        
        return {
            "personality": personality_traits,
            "background": background,
            "interests": interests
        }

    def assign_persona_to_target_llm(self, target_model: str, persona: str, 
                                    use_rich_persona: bool = True) -> Dict:
        """
        타겟 LLM에 페르소나 부여
        
        Args:
            target_model: 타겟 모델 이름
            persona: 부여할 페르소나 (예: "Software Engineer")
            use_rich_persona: 풍부한 페르소나 속성 사용 여부
        
        Returns:
            페르소나가 주입된 시스템 프롬프트와 메타데이터
        """
        # 기본 페르소나 타입 확인
        category_type = self._get_persona_category_type(persona)
        
        if use_rich_persona:
            # 풍부한 속성 생성
            attributes = self._generate_persona_attributes(persona)
            
            # 성격 특성 설명 생성
            personality_desc = []
            personality_map = {
                "openness": ("creative and open to new experiences", "practical and conventional"),
                "conscientiousness": ("organized and goal-oriented", "flexible and spontaneous"),
                "extroversion": ("outgoing and energetic", "reserved and thoughtful"),
                "agreeableness": ("cooperative and empathetic", "analytical and direct"),
                "neuroticism": ("sensitive and emotional", "calm and resilient")
            }
            
            for trait, level in attributes["personality"].items():
                if trait in personality_map:
                    desc = personality_map[trait][0] if level == "high" else personality_map[trait][1]
                    personality_desc.append(desc)
            
            # 배경 정보 문자열 생성
            background_parts = []
            if "education" in attributes["background"]:
                background_parts.append(f"education level: {attributes['background']['education']}")
            if "age_group" in attributes["background"]:
                background_parts.append(f"age: {attributes['background']['age_group']}")
            
            background_str = ", ".join(background_parts) if background_parts else "varied background"
            
            # 관심사 문자열 생성
            hobbies_str = ""
            if "hobbies" in attributes["interests"]:
                hobbies_str = ", ".join(attributes["interests"]["hobbies"][:3])
            
            system_prompt = f"""You are a {persona} with the following characteristics:

    Personality: You are {', '.join(personality_desc[:3])}.
    Background: {background_str}
    Interests: You enjoy {hobbies_str if hobbies_str else 'various activities related to your profession'}.

    When conversing:
    - Do NOT explicitly state "I am a {persona}" or reveal your role directly.
    - Naturally express your interests, experiences, and perspectives through conversation.
    - Let your personality traits influence your communication style.
    - Reference your background and interests when relevant to the topic.
    - Keep responses conversational and authentic (1-3 sentences).
    - Use casual, natural language appropriate to your character.
    """
            
            korean_prompt = f"""당신은 다음과 같은 특성을 가진 {persona}입니다:

    성격: {', '.join(personality_desc[:2])}
    배경: {background_str}
    관심사: {hobbies_str if hobbies_str else '직업과 관련된 다양한 활동'}

    대화할 때:
    - "나는 {persona}입니다"라고 명시적으로 밝히지 마세요.
    - 대화를 통해 자연스럽게 당신의 관심사, 경험, 관점을 표현하세요.
    - 성격 특성이 대화 스타일에 자연스럽게 반영되도록 하세요.
    - 주제와 관련이 있을 때 배경과 관심사를 언급하세요.
    - 짧고 자연스러운 대화체를 유지하세요 (1-3문장).
    """
        else:
            # 간단한 페르소나 프롬프트
            system_prompt = f"""You are a {persona}.
    When conversing, follow these guidelines:
    - Do not explicitly reveal your identity as a {persona}.
    - Naturally express interests, habits, and experiences that a {persona} would have.
    - Keep responses short and conversational (1-3 sentences).
    - Use casual, natural language.
    """
            
            korean_prompt = f"""당신은 {persona}입니다.
    대화할 때 다음과 같이 행동하세요:
    - 자신의 정체성을 명시적으로 드러내지 말고, 자연스럽게 대화하세요.
    - {persona}가 가질 법한 관심사, 습관, 경험을 암묵적으로 드러내세요.
    - 짧고 자연스러운 대화체를 유지하세요 (1-3문장).
    """
            attributes = None
        
        return {
            "model": target_model,
            "system_prompt": system_prompt,
            "system_prompt_ko": korean_prompt,
            "assigned_persona": persona,
            "category_type": category_type,
            "attributes": attributes,
            "use_rich_persona": use_rich_persona
        }

    # ==================== STEP 2: 대화 생성 ====================

    def generate_conversation_prompt_for_tool_llm(self, 
                                                persona_info: Dict = None,
                                                turn_count: int = 6) -> str:
        """
        도구 LLM(GPT-4)에게 줄 대화 생성 프롬프트
        
        Args:
            persona_info: 페르소나 정보 (옵션)
            turn_count: 생성할 대화 턴 수
        
        Returns:
            프롬프트 문자열
        """
        base_prompt = f"""You are a curious interviewer conducting a natural, engaging conversation.

    Generate questions following these rules for {turn_count} turns:

    1. Ask naturally about daily life, hobbies, work experiences, and interests.
    2. Use open-ended questions to encourage detailed responses.
    3. Build on previous answers with thoughtful follow-up questions.
    4. Keep questions concise and conversational (1-2 sentences).
    5. Avoid yes/no questions when possible.
    """
        
        # 페르소나 정보가 있으면 맞춤형 질문 가이드 추가
        if persona_info and persona_info.get("attributes"):
            attributes = persona_info["attributes"]
            interests = attributes.get("interests", {})
            
            interest_topics = []
            if "hobbies" in interests:
                interest_topics.extend(interests["hobbies"][:2])
            if "content_preferences" in interests:
                interest_topics.extend(interests["content_preferences"][:2])
            
            if interest_topics:
                base_prompt += f"""
    Consider exploring topics related to: {', '.join(interest_topics)}.
    """
        
        base_prompt += """
    Example questions:
    - "What did you do over the weekend?"
    - "What kind of projects are you working on these days?"
    - "How do you usually spend your free time?"
    - "What got you interested in that?"

    Now, start the conversation with your first question.
    """
        
        return base_prompt

    def conduct_conversation(
        self, 
        target_config: Dict, 
        turn_count: int = 6,
        language: str = "en"
    ) -> List[Dict]:
        """
        도구 LLM과 타겟 LLM 간 대화 진행
        
        Args:
            target_config: 타겟 LLM 설정 (assign_persona_to_target_llm 반환값)
            turn_count: 대화 턴 수
            language: 대화 언어 ("en" or "ko")
        
        Returns:
            대화 히스토리
        """
        conversation_history = []
        
        # 타겟 모델 로드
        tokenizer, model = self.load_target_model(target_config['model'])
        
        if tokenizer is None or model is None:
            print("❌ 모델 로드 실패, 대화 생성 중단")
            return []
        
        # 도구 LLM 초기 프롬프트 (페르소나 정보 포함)
        tool_llm_prompt = self.generate_conversation_prompt_for_tool_llm(
            persona_info=target_config,  # ✅ 추가됨
            turn_count=turn_count
        )
        
        for turn in range(turn_count):
            # 1. 도구 LLM이 질문 생성
            if turn == 0:
                tool_llm_input = tool_llm_prompt
            else:
                # 이전 대화 히스토리 포함
                context = "\n".join([
                    f"{msg['speaker']}: {msg['text']}" 
                    for msg in conversation_history[-4:]
                ])
                tool_llm_input = f"Previous conversation:\n{context}\n\nAsk the next question."
            
            tool_llm_response = self._call_tool_llm(tool_llm_input)  # ✅ 함수명 통일
            conversation_history.append({
                "speaker": "tool_llm",
                "text": tool_llm_response
            })
            
            print(f"[Tool LLM] {tool_llm_response}")
            
            # 2. 타겟 LLM이 답변 생성
            target_llm_response = self._call_target_llm_huggingface(
                tokenizer=tokenizer,
                model=model,
                target_config=target_config,
                user_input=tool_llm_response,
                conversation_history=conversation_history
            )
            
            conversation_history.append({
                "speaker": "target_llm",
                "text": target_llm_response
            })
            
            print(f"[Target LLM] {target_llm_response}\n")
            
            time.sleep(0.5)
        
        return conversation_history

    def _call_tool_llm(self, prompt: str, system_prompt: str = None) -> str:
        """
        도구 LLM (GPT-4) API 호출
        
        Args:
            prompt: 사용자 프롬프트
            system_prompt: 시스템 프롬프트 (옵션)
        
        Returns:
            GPT-4 응답
        """
        try:
            client = OpenAI(api_key=self.openai_api_key)
            
            messages = []
            if system_prompt:
                messages.append({"role": "system", "content": system_prompt})
            messages.append({"role": "user", "content": prompt})
            
            response = client.chat.completions.create(
                model="gpt-4o",  # ✅ 실제 존재하는 모델명
                messages=messages,
                max_tokens=150,
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"❌ GPT-4 호출 오류: {e}")
            return "[Error occurred]"

    def _call_target_llm_huggingface(
        self,
        tokenizer: AutoTokenizer,
        model: AutoModelForCausalLM,
        target_config: Dict,
        user_input: str,
        conversation_history: List[Dict]
    ) -> str:
        """
        허깅페이스 모델 호출
        
        Args:
            tokenizer: 모델 토크나이저
            model: 로드된 모델
            target_config: 타겟 설정
            user_input: 사용자 입력
            conversation_history: 대화 히스토리
        
        Returns:
            모델 응답
        """
        # 시스템 프롬프트
        system_prompt = target_config['system_prompt']
        
        # 대화 히스토리 구성 (최근 4턴)
        context = ""
        for msg in conversation_history[-4:]:
            if msg['speaker'] == 'tool_llm':
                context += f"Human: {msg['text']}\n"
            else:
                context += f"Assistant: {msg['text']}\n"
        
        # 전체 프롬프트 구성
        full_prompt = f"""{system_prompt}

    {context}Human: {user_input}
    Assistant:"""
        
        try:
            # 토크나이징
            inputs = tokenizer(
                full_prompt, 
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(self.device)
            
            # 생성
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.8,
                    top_p=0.9,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
            
            # 디코딩
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # 프롬프트 제거 후 답변만 추출
            if "Assistant:" in response:
                response = response.split("Assistant:")[-1].strip()
            
            # 너무 긴 경우 첫 2-3 문장만 사용
            sentences = response.split('.')
            if len(sentences) > 3:
                response = '.'.join(sentences[:3]) + '.'
            
            return response
            
        except Exception as e:
            print(f"❌ 타겟 LLM 생성 오류: {e}")
            return "[Error occurred]"

    # ==================== STEP 3: 브리징 추론 ====================

    def extract_bridging_relations(
        self, 
        conversation: List[Dict]
    ) -> List[Tuple[str, str, str]]:
        """
        대화에서 브리징 관계 추출 (GPT-4 기반)
        
        Args:
            conversation: 대화 히스토리
        
        Returns:
            [(anchor, anaphor, relation_type), ...]
        """
        # 타겟 LLM의 발화만 추출
        target_utterances = [
            turn['text'] for turn in conversation 
            if turn['speaker'] == 'target_llm'
        ]
        
        if not target_utterances:
            return []
        
        # GPT-4에게 브리징 관계 추출 요청
        prompt = self._create_bridging_extraction_prompt(target_utterances)
        
        bridging_json = self._call_tool_llm_for_bridging(prompt)
        
        # JSON 파싱
        try:
            # JSON 블록 추출
            if "```json" in bridging_json:
                bridging_json = bridging_json.split("```json")[1].split("```")[0]
            elif "```" in bridging_json:
                bridging_json = bridging_json.split("```")[1].split("```")[0]
            
            bridging_data = json.loads(bridging_json)
            relations = [
                (rel['anchor'], rel['anaphor'], rel['relation_type'])
                for rel in bridging_data.get('bridging_relations', [])
            ]
            return relations
        except Exception as e:
            print(f"❌ 브리징 관계 파싱 실패: {e}")
            print(f"   응답: {bridging_json[:200]}")
            return []

    def _create_bridging_extraction_prompt(self, utterances: List[str]) -> str:
        """브리징 관계 추출 프롬프트 생성 (개선 버전)"""
        utterances_text = "\n".join([f"{i+1}. {utt}" for i, utt in enumerate(utterances)])
        
        prompt = f"""Extract **Bridging Inference** relations from the following utterances.

    Utterances:
    {utterances_text}

    Bridging Inference Definition:
    - Implicit connections between concepts that require world knowledge to understand
    - Resolving references that depend on conceptual relationships rather than direct mention
    - Examples require understanding semantic frames, part-whole structures, or causal chains

    Relation Types and Examples:

    1. **Mereological (부분-전체 관계)**
    - part-of (부분): A physical or abstract part of a larger whole
        Example: "I looked into the room. The ceiling was very high."
        → room (앵커) → ceiling (대응어) [천장은 방의 일부분]
    
    - member-of (구성원): A member or element of a collection/group/set
        Example: "The class was diverse. One student stood out."
        → set/element ("class/student") [학생은 집합/요소 관계]

    2. **Frame-related (프레임 관련)**
    - instrument (도구): A tool or instrument used within an action frame
        Example: "John was murdered yesterday. The knife lay nearby."
        → murder (살인 사건) → knife (칼) [살인이라는 사건의 도구로서 칼]
    
    - theme (주제): A central theme or topic within a conceptual frame
        Example: "The conference focused on AI ethics. Privacy concerns were raised."
        → event (사건) → argument (논점) [사건/사건의 논점으로 프레임화]

    - cause-of (인과): Causal relationship between events or states
        Example: "The storm was severe. Many buildings collapsed."
        → eventuality (사건/상황) → cause (원인) [사건과 그 사건의 원인 관계]
    
    - in (공간): Spatial containment or location relationship
        Example: "We visited Paris. The museum was crowded."
        → eventuality (사건/상황) → location (장소) [사건이 발생한 장소와의 관계]
    
    - temporal (시간): Temporal relationship between events
        Example: "The meeting started. Five minutes later, the CEO arrived."
        → eventuality (사건/상황) → time (시간) [사건 발생의 시간과의 관계]

    Important Notes:
    - Focus on **implicit** relations that require inference
    - The anaphor (대응어) should NOT be directly mentioned in the same sentence as anchor (앵커)
    - Relations should reflect real-world knowledge or conceptual frames
    - Avoid marking simple co-reference (e.g., "the car... it" is NOT bridging)

    Output Requirements:
    Return ONLY valid JSON in this exact format:
    {{
    "bridging_relations": [
        {{
        "anchor": "room",
        "anaphor": "ceiling", 
        "relation_type": "part-of",
        "explanation": "천장은 방의 일부분이라는 배경지식 필요"
        }},
        {{
        "anchor": "murder",
        "anaphor": "knife",
        "relation_type": "instrument", 
        "explanation": "살인 사건 프레임에서 칼은 도구 역할"
        }}
    ]
    }}

    If no bridging relations are found, return:
    {{
    "bridging_relations": []
    }}
    """
        return prompt

    def _call_tool_llm_for_bridging(self, prompt: str) -> str:
        """브리징 추출용 GPT-4 호출"""
        try:
            client = OpenAI(api_key=self.openai_api_key)
            
            response = client.chat.completions.create(
                model="gpt-4o",  # ✅ 실제 모델명
                messages=[
                    {"role": "system", "content": "You are a linguistics expert specializing in bridging inference analysis."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=500,
                temperature=0.3
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"❌ GPT-4 브리징 추출 오류: {e}")
            return "{}"

    # ==================== STEP 4: 그래프 생성 ====================

    def build_persona_graph(self, relations: List[Tuple[str, str, str]]):
        """페르소나 그래프 구축"""
        self.persona_graph.clear()
        
        for anchor, anaphor, rel_type in relations:
            self.persona_graph.add_edge(
                anchor.lower(), 
                anaphor.lower(), 
                relation=rel_type
            )
        
        self.bridging_relations = relations
        print(f"✅ 그래프 생성 완료: {len(self.persona_graph.nodes())}개 노드, {len(self.persona_graph.edges())}개 엣지")

    def get_central_nodes(self, top_k: int = 3) -> List[Tuple[str, float]]:
        """그래프 중심 노드 추출 (PageRank 기반)"""
        if len(self.persona_graph) == 0:
            print("⚠️  그래프가 비어있습니다.")
            return []
        
        try:
            pagerank = nx.pagerank(self.persona_graph)
            sorted_nodes = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)
            return sorted_nodes[:top_k]
        except Exception as e:
            print(f"❌ PageRank 계산 실패: {e}")
            return []
        
    # ==================== STEP 5: 페르소나 추론 ====================

    def infer_persona_from_graph(self) -> str:
        """
        그래프 중심 노드로부터 페르소나 추론 (GPT-4 사용)
        
        Returns:
            추론된 페르소나
        """
        central_nodes = self.get_central_nodes(top_k=5)
        
        if not central_nodes:
            return "Unknown"
        
        # GPT-4에게 페르소나 추론 요청
        nodes_text = ", ".join([node for node, _ in central_nodes])
        
        # 페르소나 카테고리를 영문으로 추출
        persona_list = "\n".join([f"- {persona}" for persona in self.persona_categories])
        
        prompt = f"""The following are key concepts extracted from a conversation through bridging inference analysis:
    {nodes_text}

    Based on these concepts and their relationships, select ONE persona category from the following list:
    {persona_list}

    Answer with only the most appropriate persona name from the list above.
    """
        
        inferred_persona = self._call_tool_llm(prompt).strip()
        
        # 카테고리 리스트에서 가장 유사한 것 찾기
        for category in self.persona_categories:
            # 영문 부분만 추출해서 비교
            category_en = category.split('(')[-1].replace(')', '').strip() if '(' in category else category
            if category_en.lower() in inferred_persona.lower() or category in inferred_persona:
                return category
        
        # 매칭 실패 시 원본 반환
        return inferred_persona

    # ==================== STEP 6: 비교 분석 (Baseline) ====================

    def baseline_llm_direct_inference(self, conversation: List[Dict]) -> str:
        """
        베이스라인: 브리징 그래프 없이 LLM이 대화 내용만 보고 직접 페르소나 추론
        
        Args:
            conversation: 대화 히스토리
        
        Returns:
            추론된 페르소나
        """
        # 타겟 LLM 발화만 추출
        target_utterances = [
            turn['text'] for turn in conversation 
            if turn['speaker'] == 'target_llm'
        ]
        
        if not target_utterances:
            return "Unknown"
        
        # 대화 내용 포맷팅
        conversation_text = "\n".join([
            f"{i+1}. {utt}" for i, utt in enumerate(target_utterances)
        ])
        
        # 페르소나 카테고리 리스트
        persona_list = "\n".join([f"- {persona}" for persona in self.persona_categories])
        
        # GPT-4에게 직접 추론 요청 (브리징 정보 없이)
        prompt = f"""Read the following conversation responses and infer the speaker's persona.

    Conversation responses:
    {conversation_text}

    Based on the conversation content, select ONE persona category from the following list:
    {persona_list}

    Answer with only the most appropriate persona name from the list above.
    Do not use any external knowledge or bridging inference - only use what is explicitly stated in the conversation.
    """
        
        try:
            inferred_persona = self._call_tool_llm(prompt).strip()
            
            # 카테고리 리스트에서 가장 유사한 것 찾기
            for category in self.persona_categories:
                category_en = category.split('(')[-1].replace(')', '').strip() if '(' in category else category
                if category_en.lower() in inferred_persona.lower() or category in inferred_persona:
                    return category
            
            return inferred_persona
            
        except Exception as e:
            print(f"❌ Baseline 추론 실패: {e}")
            return "Unknown (Error)"

    # ==================== STEP 7: 시각화 & 리포트 ====================

    def visualize_graph(self, save_path: str = 'persona_graph.png'):
        """그래프 시각화"""
        if len(self.persona_graph) == 0:
            print("❌ 그래프가 비어있습니다.")
            return
        
        plt.figure(figsize=(12, 8))
        pos = nx.spring_layout(self.persona_graph, k=0.5, seed=42)
        
        # 노드 크기 (PageRank 기반)
        try:
            pagerank = nx.pagerank(self.persona_graph)
            node_sizes = [pagerank.get(node, 0.01) * 5000 for node in self.persona_graph.nodes()]
        except:
            node_sizes = [1000] * len(self.persona_graph.nodes())
        
        nx.draw_networkx_nodes(
            self.persona_graph, pos,
            node_size=node_sizes,
            node_color='lightblue',
            alpha=0.7
        )
        
        nx.draw_networkx_labels(
            self.persona_graph, pos,
            font_size=10,
            font_weight='bold'
        )
        
        nx.draw_networkx_edges(
            self.persona_graph, pos,
            edge_color='gray',
            arrows=True,
            arrowsize=20,
            width=2
        )
        
        # 엣지 라벨 (관계 타입)
        edge_labels = nx.get_edge_attributes(self.persona_graph, 'relation')
        nx.draw_networkx_edge_labels(
            self.persona_graph, pos,
            edge_labels=edge_labels,
            font_size=8
        )
        
        plt.title("Persona Discourse Graph (Bridging Inference)", fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✅ 그래프 저장: {save_path}")

    def generate_evaluation_report(
        self, 
        assigned_persona: str,
        inferred_persona: str,
        baseline_persona: str,
        conversation: List[Dict],
        model_name: str
    ) -> str:
        """평가 리포트 생성"""
        
        # 정확도 계산 (영문 페르소나명 기준)
        def extract_persona_name(persona_str):
            """페르소나 문자열에서 핵심 이름 추출"""
            if '(' in persona_str:
                # "여행자 (Traveler)" 형식인 경우 영문 추출
                return persona_str.split('(')[-1].replace(')', '').strip().lower()
            return persona_str.strip().lower()
        
        assigned_name = extract_persona_name(assigned_persona)
        inferred_name = extract_persona_name(inferred_persona)
        baseline_name = extract_persona_name(baseline_persona)
        
        bridging_correct = assigned_name in inferred_name or inferred_name in assigned_name
        baseline_correct = assigned_name in baseline_name or baseline_name in assigned_name
        
        report = f"""
    {'='*70}
    LLM Persona Discovery Evaluation Report
    {'='*70}

    Target Model: {model_name}

    1. Ground Truth (부여된 페르소나):
    → {assigned_persona}

    2. Bridging Inference Method (제안 방법 - 그래프 기반):
    → 추론된 페르소나: {inferred_persona}
    → 정확도: {'✅ 일치' if bridging_correct else '❌ 불일치'}

    3. Baseline (LLM 직접 추론 - 그래프 없이):
    → 추론된 페르소나: {baseline_persona}
    → 정확도: {'✅ 일치' if baseline_correct else '❌ 불일치'}

    4. Method Comparison:
    • Bridging Method: {'정확' if bridging_correct else '부정확'}
    • Baseline Method: {'정확' if baseline_correct else '부정확'}
    • Winner: {'🏆 Bridging Method' if bridging_correct and not baseline_correct else '🏆 Baseline Method' if baseline_correct and not bridging_correct else '🤝 Both Correct' if bridging_correct and baseline_correct else '❌ Both Incorrect'}

    5. 브리징 관계 (총 {len(self.bridging_relations)}개):
    """
        
        if self.bridging_relations:
            for i, (anchor, anaphor, rel_type) in enumerate(self.bridging_relations[:10], 1):
                report += f"   {i}. {anchor} → {anaphor} [{rel_type}]\n"
            
            if len(self.bridging_relations) > 10:
                report += f"   ... (and {len(self.bridging_relations) - 10} more)\n"
        else:
            report += "   (브리징 관계 없음)\n"
        
        report += f"\n6. 그래프 중심 노드 (Top 5):\n"
        central_nodes = self.get_central_nodes(5)
        if central_nodes:
            for node, score in central_nodes:
                report += f"   • {node}: {score:.4f}\n"
        else:
            report += "   (그래프 비어있음)\n"
        
        report += f"\n7. 대화 샘플 (처음 4턴):\n"
        sample_turns = conversation[:8]  # 4개 질문-응답 쌍
        for i, turn in enumerate(sample_turns):
            speaker = "🤖 Interviewer" if turn['speaker'] == 'tool_llm' else "💬 Target LLM"
            report += f"   [{speaker}] {turn['text']}\n"
        
        report += f"\n{'='*70}\n"
        
        return report

    def cleanup_models(self):
        """모델 메모리 정리"""
        for model_name, (tokenizer, model) in self.loaded_models.items():
            del tokenizer
            del model
        self.loaded_models.clear()
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        print("🧹 모델 메모리 정리 완료")

    def save_results_to_json(
        self,
        assigned_persona: str,
        inferred_persona: str,
        baseline_persona: str,
        conversation: List[Dict],
        model_name: str,
        save_path: str = "results.json"
    ):
        """결과를 JSON 파일로 저장"""
        
        def extract_persona_name(persona_str):
            if '(' in persona_str:
                return persona_str.split('(')[-1].replace(')', '').strip()
            return persona_str.strip()
        
        assigned_name = extract_persona_name(assigned_persona)
        inferred_name = extract_persona_name(inferred_persona)
        baseline_name = extract_persona_name(baseline_persona)
        
        results = {
            "model_name": model_name,
            "assigned_persona": assigned_persona,
            "inferred_persona": inferred_persona,
            "baseline_persona": baseline_persona,
            "bridging_correct": assigned_name.lower() in inferred_name.lower(),
            "baseline_correct": assigned_name.lower() in baseline_name.lower(),
            "bridging_relations": [
                {
                    "anchor": anchor,
                    "anaphor": anaphor,
                    "relation_type": rel_type
                }
                for anchor, anaphor, rel_type in self.bridging_relations
            ],
            "central_nodes": [
                {"node": node, "score": float(score)}
                for node, score in self.get_central_nodes(10)
            ],
            "conversation": conversation
        }
        
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        print(f"✅ 결과 저장: {save_path}")

    # ==================== 전체 파이프라인 실행 ====================

def run_full_pipeline(
    target_model_name: str, 
    persona: str,
    api_key: Optional[str] = None,
    turn_count: int = 6,
    use_rich_persona: bool = True,
    save_json: bool = True
):
    """
    전체 파이프라인 실행
    
    Args:
        target_model_name: 타겟 LLM 모델명
        persona: 부여할 페르소나
        api_key: OpenAI API 키 (None이면 환경변수에서 로드)
        turn_count: 대화 턴 수
        use_rich_persona: 풍부한 페르소나 속성 사용 여부
        save_json: JSON 결과 저장 여부
    
    Returns:
        실험 결과 딕셔너리
    """
    
    # API 키 로드
    if api_key is None:
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("❌ OPENAI_API_KEY not found in environment variables")
    
    persona_path = '/Users/jisu/Desktop/2025Workshop/persona_schema.json'
    pipeline = LLMPersonaDiscoveryPipeline(api_key, persona_path)
    
    print("\n" + "=" * 70)
    print(f"🚀 Starting Pipeline: {target_model_name} as '{persona}'")
    print("=" * 70)
    
    # 1. 페르소나 부여
    print("\n📋 STEP 1: Assigning Persona to Target LLM")
    print("-" * 70)
    
    target_config = pipeline.assign_persona_to_target_llm(
        target_model=target_model_name,
        persona=persona,
        use_rich_persona=use_rich_persona
    )
    
    print(f"✅ Assigned Persona: {persona}")
    if use_rich_persona and target_config.get('attributes'):
        print(f"   Category: {target_config['category_type']}")
        attrs = target_config['attributes']
        if 'personality' in attrs:
            print(f"   Personality: {list(attrs['personality'].items())[:2]}")
    
    # 2. 대화 생성
    print("\n💬 STEP 2: Generating Conversation")
    print("-" * 70)
    
    conversation = pipeline.conduct_conversation(
        target_config=target_config,
        turn_count=turn_count,
        language="en"
    )
    
    if not conversation:
        print("❌ 대화 생성 실패")
        return None
    
    print(f"✅ Generated {len(conversation)} conversation turns")
    
    # 3. 브리징 관계 추출
    print("\n🔗 STEP 3: Extracting Bridging Relations")
    print("-" * 70)
    
    bridging_relations = pipeline.extract_bridging_relations(conversation)
    
    if bridging_relations:
        print(f"✅ Extracted {len(bridging_relations)} bridging relations:")
        for anchor, anaphor, rel_type in bridging_relations[:5]:  # 처음 5개만 표시
            print(f"   {anchor} → {anaphor} [{rel_type}]")
        if len(bridging_relations) > 5:
            print(f"   ... and {len(bridging_relations) - 5} more")
    else:
        print("⚠️  No bridging relations found")
    
    # 4. 그래프 구축
    print("\n📊 STEP 4: Building Persona Graph")
    print("-" * 70)
    
    pipeline.build_persona_graph(bridging_relations)
    
    # 5. 페르소나 추론 (제안 방법)
    print("\n🎯 STEP 5: Inferring Persona (Bridging Method)")
    print("-" * 70)
    
    inferred_persona = pipeline.infer_persona_from_graph()
    print(f"✅ Inferred Persona: {inferred_persona}")
    
    # 6. 베이스라인 비교
    print("\n📈 STEP 6: Baseline Comparison (LLM Direct Inference)")
    print("-" * 70)
    
    baseline_persona = pipeline.baseline_llm_direct_inference(conversation)
    print(f"✅ Baseline Persona: {baseline_persona}")
    
    # 7. 평가 리포트
    print("\n📄 STEP 7: Generating Evaluation Report")
    print("-" * 70)
    
    report = pipeline.generate_evaluation_report(
        assigned_persona=persona,
        inferred_persona=inferred_persona,
        baseline_persona=baseline_persona,
        conversation=conversation,
        model_name=target_model_name
    )
    print(report)
    
    # 8. 시각화
    print("\n🎨 STEP 8: Visualizing Graph")
    print("-" * 70)
    
    safe_model_name = target_model_name.replace('/', '_')
    safe_persona_name = persona.split('(')[0].strip().replace(' ', '_')
    graph_path = f'persona_graph_{safe_model_name}_{safe_persona_name}.png'
    
    if len(pipeline.persona_graph) > 0:
        pipeline.visualize_graph(graph_path)
    else:
        print("⚠️  그래프가 비어있어 시각화를 건너뜁니다")
    
    # 9. 결과 저장
    result = {
        'model': target_model_name,
        'assigned_persona': persona,
        'inferred_persona': inferred_persona,
        'baseline_persona': baseline_persona,
        'bridging_correct': _check_persona_match(persona, inferred_persona),
        'baseline_correct': _check_persona_match(persona, baseline_persona),
        'num_bridging_relations': len(bridging_relations),
        'num_graph_nodes': len(pipeline.persona_graph.nodes()),
        'num_graph_edges': len(pipeline.persona_graph.edges())
    }
    
    # JSON 저장
    if save_json:
        json_path = f'results_{safe_model_name}_{safe_persona_name}.json'
        pipeline.save_results_to_json(
            assigned_persona=persona,
            inferred_persona=inferred_persona,
            baseline_persona=baseline_persona,
            conversation=conversation,
            model_name=target_model_name,
            save_path=json_path
        )
    
    # 10. 메모리 정리
    print("\n🧹 STEP 9: Cleaning Up")
    print("-" * 70)
    pipeline.cleanup_models()
    
    return result


def _check_persona_match(assigned: str, inferred: str) -> bool:
    """
    페르소나 매칭 여부 확인
    
    Args:
        assigned: 부여된 페르소나
        inferred: 추론된 페르소나
    
    Returns:
        매칭 여부
    """
    def extract_name(persona_str):
        if '(' in persona_str:
            return persona_str.split('(')[-1].replace(')', '').strip().lower()
        return persona_str.strip().lower()
    
    assigned_name = extract_name(assigned)
    inferred_name = extract_name(inferred)
    
    return assigned_name in inferred_name or inferred_name in assigned_name


def run_all_experiments(
    models: List[str] = None,
    personas: List[str] = None,
    api_key: Optional[str] = None,
    turn_count: int = 6,
    use_rich_persona: bool = True
):
    """
    모든 실험 실행
    
    Args:
        models: 실험할 모델 리스트 (None이면 기본값 사용)
        personas: 실험할 페르소나 리스트 (None이면 기본값 사용)
        api_key: OpenAI API 키 (None이면 환경변수에서 로드)
        turn_count: 대화 턴 수
        use_rich_persona: 풍부한 페르소나 사용 여부
    
    Returns:
        실험 결과 리스트
    """
    
    # API 키 로드
    if api_key is None:
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("❌ OPENAI_API_KEY not found in environment variables")
    
    # 기본 모델 리스트
    if models is None:
        models = [
            "meta-llama/Llama-3.2-1B",
            "Qwen/Qwen2.5-0.5B",
            # "deepseek-ai/deepseek-vl2-tiny"  # VL 모델은 제외
        ]
    
    # 기본 페르소나 리스트 (JSON에서 일부 추출)
    if personas is None:
        # 파이프라인 초기화해서 페르소나 가져오기
        temp_pipeline = LLMPersonaDiscoveryPipeline(api_key)
        all_personas = temp_pipeline.persona_categories
        
        # 각 카테고리에서 균형있게 선택
        professional = temp_pipeline.get_persona_by_category("professional")
        technical = temp_pipeline.get_persona_by_category("technical_management")
        general = temp_pipeline.get_persona_by_category("general_role")
        
        personas = []
        if professional:
            personas.append(professional[0])  # 첫 번째 전문직
        if technical:
            personas.append(technical[0])  # 첫 번째 기술직
        if general:
            personas.append(general[0])  # 첫 번째 일반직
        
        # 페르소나가 3개 미만이면 추가
        if len(personas) < 3 and len(all_personas) >= 3:
            for p in all_personas:
                if p not in personas:
                    personas.append(p)
                if len(personas) >= 3:
                    break
        
        print(f"✅ 선택된 페르소나: {personas}")
    
    results = []
    total_experiments = len(models) * len(personas)
    current_experiment = 0
    
    print("\n" + "=" * 70)
    print(f"🧪 Starting {total_experiments} Experiments")
    print(f"   Models: {len(models)}")
    print(f"   Personas: {len(personas)}")
    print("=" * 70)
    
    for model in models:
        for persona in personas:
            current_experiment += 1
            
            print("\n\n" + "=" * 70)
            print(f"🧪 Experiment {current_experiment}/{total_experiments}: {model} × {persona}")
            print("=" * 70)
            
            try:
                result = run_full_pipeline(
                    target_model_name=model,
                    persona=persona,
                    api_key=api_key,
                    turn_count=turn_count,
                    use_rich_persona=use_rich_persona,
                    save_json=True
                )
                
                if result:
                    results.append(result)
                    print(f"✅ Experiment {current_experiment} completed successfully")
                else:
                    print(f"⚠️  Experiment {current_experiment} returned no result")
                    
            except Exception as e:
                print(f"❌ Experiment {current_experiment} failed: {e}")
                import traceback
                traceback.print_exc()
            
            # API 속도 제한 방지
            time.sleep(3)
    
    # 전체 결과 요약
    print("\n\n" + "=" * 70)
    print("📊 FINAL RESULTS SUMMARY")
    print("=" * 70)
    
    if not results:
        print("❌ No successful experiments")
        return results
    
    bridging_correct = sum(r['bridging_correct'] for r in results)
    baseline_correct = sum(r['baseline_correct'] for r in results)
    total = len(results)
    
    bridging_accuracy = (bridging_correct / total) * 100
    baseline_accuracy = (baseline_correct / total) * 100
    improvement = bridging_accuracy - baseline_accuracy
    
    print(f"\nTotal Experiments: {total}")
    print(f"Successful: {len(results)}")
    print(f"\n{'Method':<25} {'Correct':<10} {'Accuracy'}")
    print("-" * 50)
    print(f"{'Bridging Inference':<25} {bridging_correct}/{total:<10} {bridging_accuracy:.1f}%")
    print(f"{'Baseline (LLM Direct)':<25} {baseline_correct}/{total:<10} {baseline_accuracy:.1f}%")
    print("-" * 50)
    print(f"{'Improvement':<25} {'':<10} {improvement:+.1f}%")
    
    # 모델별 결과
    print("\n📊 Results by Model:")
    for model in models:
        model_results = [r for r in results if r['model'] == model]
        if model_results:
            bridging_acc = sum(r['bridging_correct'] for r in model_results) / len(model_results) * 100
            baseline_acc = sum(r['baseline_correct'] for r in model_results) / len(model_results) * 100
            print(f"  {model}")
            print(f"    Bridging: {bridging_acc:.1f}% | Baseline: {baseline_acc:.1f}%")
    
    # 페르소나별 결과
    print("\n📊 Results by Persona:")
    for persona in personas:
        persona_results = [r for r in results if r['assigned_persona'] == persona]
        if persona_results:
            bridging_acc = sum(r['bridging_correct'] for r in persona_results) / len(persona_results) * 100
            baseline_acc = sum(r['baseline_correct'] for r in persona_results) / len(persona_results) * 100
            print(f"  {persona}")
            print(f"    Bridging: {bridging_acc:.1f}% | Baseline: {baseline_acc:.1f}%")
    
    # 전체 결과를 JSON으로 저장
    summary_path = "experiment_summary.json"
    with open(summary_path, 'w', encoding='utf-8') as f:
        json.dump({
            "total_experiments": total,
            "bridging_accuracy": bridging_accuracy,
            "baseline_accuracy": baseline_accuracy,
            "improvement": improvement,
            "detailed_results": results
        }, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ Summary saved to: {summary_path}")
    print("=" * 70 + "\n")
    
    return results



In [6]:
# ==================== Jupyter Notebook용 실행 코드 ====================

# 환경변수에서 API 키 로드
import os
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("❌ Error: OPENAI_API_KEY environment variable not set")
    print("   Please set it in the notebook:")
    print("   import os")
    print("   os.environ['OPENAI_API_KEY'] = 'your-api-key'")
else:
    print("✅ API Key loaded successfully")

# ========== 옵션 1: 단일 실험 실행 ==========
result = run_full_pipeline(
    target_model_name="Qwen/Qwen2.5-0.5B",
    persona="Doctor",
    api_key=api_key,
    turn_count=6,
    use_rich_persona=True,
    save_json=True
)

print(f"\n✅ Single experiment completed: {result}")

✅ API Key loaded successfully
⚠️  스키마 파일을 찾을 수 없습니다: persona_schema.json
📊 추출된 페르소나: 5개
🖥️  사용 디바이스: cpu
📋 로드된 페르소나 카테고리 수: 5

🚀 Starting Pipeline: Qwen/Qwen2.5-0.5B as 'Doctor'

📋 STEP 1: Assigning Persona to Target LLM
----------------------------------------------------------------------
✅ Assigned Persona: Doctor
   Category: professional
   Personality: []

💬 STEP 2: Generating Conversation
----------------------------------------------------------------------
📥 모델 다운로드 중: Qwen/Qwen2.5-0.5B
✅ 모델 로드 완료: Qwen/Qwen2.5-0.5B
[Tool LLM] What does a typical day look like for you, and how do you usually start your mornings?
[Target LLM] My typical day is filled with different aspects of my life, and I usually start my mornings with a quick morning routine that helps me get ready for the day. I usually wake up around 6:30am, grab my favorite coffee and breakfast, and then do my morning exercises to get my body moving and energized. I usually have a short lunch and then head out to the park

In [ ]:
# ==================== 메인 실행 ====================

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description="LLM Persona Discovery Pipeline")
    parser.add_argument("--model", type=str, help="Single model to test")
    parser.add_argument("--persona", type=str, help="Single persona to test")
    parser.add_argument("--turn_count", type=int, default=6, help="Number of conversation turns")
    parser.add_argument("--all", action="store_true", help="Run all experiments")
    parser.add_argument("--rich_persona", action="store_true", default=True, help="Use rich persona attributes")
    
    args = parser.parse_args()
    
    # 환경변수에서 API 키 로드
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("❌ Error: OPENAI_API_KEY environment variable not set")
        print("   Please set it using: export OPENAI_API_KEY='your-api-key'")
        exit(1)
    
    if args.all:
        # 모든 실험 실행
        results = run_all_experiments(
            api_key=api_key,
            turn_count=args.turn_count,
            use_rich_persona=args.rich_persona
        )
    elif args.model and args.persona:
        # 단일 실험 실행
        result = run_full_pipeline(
            target_model_name=args.model,
            persona=args.persona,
            api_key=api_key,
            turn_count=args.turn_count,
            use_rich_persona=args.rich_persona
        )
        print(f"\n✅ Single experiment completed: {result}")
    else:
        print("❌ Please specify --all or both --model and --persona")
        parser.print_help()